In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx

In [2]:
churches = ['mossleyhill','sthelens','gateway','stoneycroftsalvationarmy']

In [3]:
location = {'mossleyhill' : 'L18 8DB',
            'sthelens' : 'WA10 2DT',
            'gateway' : 'L24 9HJ',
            'stoneycroftsalvationarmy': 'L13 3BT'}

In [4]:
mile = 1609 # metres

In [5]:
pcdlatlng = pd.read_csv('csv/pcdlatlng.csv')

In [6]:
def get_coords(postcode):
    "Lookup coordinates from conversion table."
    row = pcdlatlng.loc[pcdlatlng.postcode==postcode].iloc[0]
    return row.latitude, row.longitude

In [7]:
def get_street_names(coords, miles):
    neighbourhood_graph = ox.project_graph(ox.graph_from_point(coords, distance=miles*mile, network_type='drive'))
    ints = ox.clean_intersections(neighbourhood_graph)

    gdf = gpd.GeoDataFrame(ints, columns=['geometry'], crs=neighbourhood_graph.graph['crs'])
    X = gdf['geometry'].map(lambda pt: pt.coords[0][0])
    Y = gdf['geometry'].map(lambda pt: pt.coords[0][1])

    nodes = ox.get_nearest_nodes(neighbourhood_graph, X, Y, method='kdtree')
    nearby_streets = {'Rose Lane'}
    for n in nodes:
        for nbr in nx.neighbors(neighbourhood_graph, n):
            for d in neighbourhood_graph.get_edge_data(n, nbr).values():
                if 'name' in d:
                    if type(d['name']) == str:
                        nearby_streets.add(d['name'])
                    elif type(d['name']) == list:
                        for name in d['name']:
                            nearby_streets.add(name)
                    else:
                        pass
                else:
                    pass

    return nearby_streets

In [8]:
streets = {}
for church in churches:
    streets[church] = get_street_names(get_coords(location[church]), 1)

In [9]:
for church in churches:
    print(church)
    print(f'{len(streets[church])} streets within 1 mile')
    print(streets[church])
    print('\n')

mossleyhill
391 streets within 1 mile
{'Alma Road', 'Lochmore Road', 'Elmsdale Road', 'Templemore Avenue', 'Queensdale Road', 'Chequers Gardens', 'Zetland Road', 'Mosslea Park', 'Briarwood Road', 'Barndale Road', 'Melbreck Road', 'Fulwood Drive', 'Cromer Road', 'Molyneux Road', 'Pitville Terrace', 'Friarsgate Close', 'The Quadrangle', 'Kylemore Avenue', 'Oakland Road', 'Corndale Road', 'Mossville Road', 'Coventry Road', 'Heydean Road', 'Menlove Gardens South', 'Lammermoor Road', 'Marlcroft Drive', 'Allerton Road', 'Kirkmore Road', 'Abbotts Close', 'Rutherford Road', 'Ranelagh Drive South', 'Erylmore Road', 'South Sudley Road', 'Elder Gardens', 'Oldfield Road', 'Greenhill Close', 'Mossley Avenue', 'Menlove Avenue', 'Montclair Drive', 'Silverton Road', 'Cyril Grove', 'Boxmoor Road', 'Nithsdale Road', 'Mareth Close', 'Norbury Avenue', 'Jericho Lane', 'Langford Road', 'Greenbank Lane', 'Fallowfield Road', 'Martin Close', 'Verney Crescent South', 'Drewell Road', 'Pinemore Road', 'Cramond Av